Favorite books on PySpark:
* Spark: The Definitive Guide https://learning.oreilly.com/library/view/spark-the-definitive/9781491912201/
* Learning Spark https://learning.oreilly.com/library/view/learning-spark-2nd/9781492050032/

Documentation:
* https://spark.apache.org/docs/latest/api/python/index.html

Articles:
* Best practices for caching in Spark SQL https://towardsdatascience.com/best-practices-for-caching-in-spark-sql-b22fb0f02d34
* Be in charge of Query Execution in Spark SQL https://towardsdatascience.com/be-in-charge-of-query-execution-in-spark-sql-c83d1e16b9b8
* Spark DataFrame Cache and Persist Explained https://sparkbyexamples.com/spark/spark-dataframe-cache-and-persist-explained/
* Should I always cache my RDD's and DataFrames? https://forums.databricks.com/questions/271/should-i-always-cache-my-rdds.html
* PySpark Dataframe Basics https://changhsinlee.com/pyspark-dataframe-basics/
* How to Turn Python Functions into PySpark Functions (UDF) https://changhsinlee.com/pyspark-udf/
* Approximate Algorithms in Apache Spark: HyperLogLog and Quantiles https://databricks.com/blog/2016/05/19/approximate-algorithms-in-apache-spark-hyperloglog-and-quantiles.html
* Interactive Audience Analytics With Apache Spark and HyperLogLog https://databricks.com/blog/2015/10/13/interactive-audience-analytics-with-apache-spark-and-hyperloglog.html
* Introducing Pandas UDF for PySpark https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html
* Spark by Examples https://sparkbyexamples.com/

Data used in this notebook is from "Sberbank Russian Housing Market" competition on Kaggle: https://www.kaggle.com/c/sberbank-russian-housing-market

In [100]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import os

# Creating Spark Session

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

# Quick Overview

In [74]:
SBERBANK_BASE_DIR = "/Users/anton/datasets/sberbank_russian_housing_market"
SBERBANK_PATH_FULL = os.path.join(SBERBANK_BASE_DIR, "train.csv")
SBERBANK_PATH_SUBSET = os.path.join(SBERBANK_BASE_DIR, "train_subset.csv")

df_full = spark.read.option("inferSchema", "true").option("header", "true").csv(SBERBANK_PATH_FULL)
df_full.printSchema()

root
 |-- id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- full_sq: integer (nullable = true)
 |-- life_sq: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- max_floor: string (nullable = true)
 |-- material: string (nullable = true)
 |-- build_year: string (nullable = true)
 |-- num_room: string (nullable = true)
 |-- kitch_sq: string (nullable = true)
 |-- state: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- sub_area: string (nullable = true)
 |-- area_m: double (nullable = true)
 |-- raion_popul: integer (nullable = true)
 |-- green_zone_part: double (nullable = true)
 |-- indust_part: double (nullable = true)
 |-- children_preschool: integer (nullable = true)
 |-- preschool_quota: string (nullable = true)
 |-- preschool_education_centers_raion: integer (nullable = true)
 |-- children_school: integer (nullable = true)
 |-- school_quota: string (nullable = true)
 |-- school_education_centers_raion: integer (nu

In [75]:
with pd.option_context('display.max_columns', None): 
    display(df_full.limit(5).toPandas())

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_av

In [76]:
df = df_full.select("timestamp",       # date of transaction
                    "full_sq",         # total area
                    "life_sq",         # living area
                    "floor",
                    "sub_area",        # district
                    "green_zone_part",
                    "indust_part",
                    "price_doc").      # sale price

In [82]:
df.agg({"price_doc": "max"}).show()

+--------------+
|max(price_doc)|
+--------------+
|     111111112|
+--------------+



In [83]:
df.agg({"price_doc": "min"}).show()

+--------------+
|min(price_doc)|
+--------------+
|        100000|
+--------------+



In [78]:
df.write.mode("overwrite").option("header", "true").csv(SBERBANK_PATH_SUBSET)

In [79]:
df = spark.read.option("inferSchema", "true").option("header", "true").csv(SBERBANK_PATH_SUBSET)
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- full_sq: integer (nullable = true)
 |-- life_sq: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- sub_area: string (nullable = true)
 |-- green_zone_part: double (nullable = true)
 |-- indust_part: double (nullable = true)
 |-- price_doc: integer (nullable = true)



# Creating and Reading Data

In [85]:
numbers = spark.range(10)
numbers.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [256]:
spark.range(1).selectExpr("'Hello' as key", "0 as value").show()

+-----+-----+
|  key|value|
+-----+-----+
|Hello|    0|
+-----+-----+



In [87]:
import tempfile
import os

with tempfile.NamedTemporaryFile() as f:
    f.write(b'1\t"Anna"\n2\tBoris')
    f.flush()
    os.fsync(f)

    schema = StructType([
        StructField("ID", LongType(), nullable=False),
        StructField("Name", StringType(), nullable=False)
    ])

    spark.read.format("csv").schema(schema).option('sep', '\t').load(f.name).show()

+---+-----+
| ID| Name|
+---+-----+
|  1| Anna|
|  2|Boris|
+---+-----+



In [112]:
schema = StructType([
    StructField("Name", StringType(), nullable=False),
    StructField("ID", LongType(), nullable=False)
])
row = Row("Hello", 1)
spark.createDataFrame([row], schema).show()

+-----+---+
| Name| ID|
+-----+---+
|Hello|  1|
+-----+---+



In [286]:
spark.createDataFrame([
    (10, "Bob"),
    (20, "Ann")]).toDF("id", "name").show()

+---+----+
| id|name|
+---+----+
| 10| Bob|
| 20| Ann|
+---+----+



In [315]:
df = spark \
  .read \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .csv(SBERBANK_PATH_SUBSET)

In [316]:
df.createOrReplaceTempView("df")
spark.sql("SELECT count(*) FROM df").show()

+--------+
|count(1)|
+--------+
|   30471|
+--------+



# Showing Data

In [88]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- full_sq: integer (nullable = true)
 |-- life_sq: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- sub_area: string (nullable = true)
 |-- green_zone_part: double (nullable = true)
 |-- indust_part: double (nullable = true)
 |-- price_doc: integer (nullable = true)



In [95]:
df.schema

StructType(List(StructField(timestamp,TimestampType,true),StructField(full_sq,IntegerType,true),StructField(life_sq,StringType,true),StructField(floor,StringType,true),StructField(sub_area,StringType,true),StructField(green_zone_part,DoubleType,true),StructField(indust_part,DoubleType,true),StructField(price_doc,IntegerType,true)))

In [89]:
df.schema.names

['timestamp',
 'full_sq',
 'life_sq',
 'floor',
 'sub_area',
 'green_zone_part',
 'indust_part',
 'price_doc']

In [127]:
df.count() # number of rows

30471

In [90]:
df.take(2)

[Row(timestamp=datetime.datetime(2014, 3, 27, 0, 0), full_sq=35, life_sq='19', floor='3', sub_area='Ivanovskoe', green_zone_part=0.512707469, indust_part=0.000169676, price_doc=5300000),
 Row(timestamp=datetime.datetime(2014, 3, 27, 0, 0), full_sq=46, life_sq='NA', floor='12', sub_area='Tverskoe', green_zone_part=0.065444314, indust_part=7.81528e-05, price_doc=4648932)]

In [91]:
df.show(n=2)

+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
|          timestamp|full_sq|life_sq|floor|  sub_area|green_zone_part|indust_part|price_doc|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
|2014-03-27 00:00:00|     35|     19|    3|Ivanovskoe|    0.512707469| 1.69676E-4|  5300000|
|2014-03-27 00:00:00|     46|     NA|   12|  Tverskoe|    0.065444314| 7.81528E-5|  4648932|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
only showing top 2 rows



In [92]:
df.limit(2).collect()

[Row(timestamp=datetime.datetime(2014, 3, 27, 0, 0), full_sq=35, life_sq='19', floor='3', sub_area='Ivanovskoe', green_zone_part=0.512707469, indust_part=0.000169676, price_doc=5300000),
 Row(timestamp=datetime.datetime(2014, 3, 27, 0, 0), full_sq=46, life_sq='NA', floor='12', sub_area='Tverskoe', green_zone_part=0.065444314, indust_part=7.81528e-05, price_doc=4648932)]

In [99]:
df.first()

Row(timestamp=datetime.datetime(2014, 3, 27, 0, 0), full_sq=35, life_sq='19', floor='3', sub_area='Ivanovskoe', green_zone_part=0.512707469, indust_part=0.000169676, price_doc=5300000)

In [93]:
df.limit(2).toPandas()

,timestamp,full_sq,life_sq,floor,sub_area,green_zone_part,indust_part,price_doc
0,2014-03-27,35,19,3,Ivanovskoe,0.512707,0.000170,5300000
1,2014-03-27,46,NA,12,Tverskoe,0.065444,0.000078,4648932


# Operations

## Columns and expressions

> To Spark, columns are logical constructions that simply represent a value computed on a per-record basis by means of an expression. This means that to have a real value for a column, we need to have a row; and to have a row, we need to have a DataFrame. You cannot manipulate an individual column outside the context of a DataFrame

From "Spark: The Definitive Guide"

In [94]:
col('abc'), column('timestamp')

(Column<b'abc'>, Column<b'timestamp'>)

In [98]:
expr('full_sq > 35')

Column<b'(full_sq > 35)'>

## Basics

In [116]:
df.select("timestamp", "price_doc").show(2)

+-------------------+---------+
|          timestamp|price_doc|
+-------------------+---------+
|2014-03-27 00:00:00|  5300000|
|2014-03-27 00:00:00|  4648932|
+-------------------+---------+
only showing top 2 rows



In [119]:
df.select(expr("green_zone_part * 100").alias("green_zone_percent"), expr("price_doc / 75 as price_usd")).show(2)

+------------------+-----------------+
|green_zone_percent|        price_usd|
+------------------+-----------------+
|        51.2707469|70666.66666666667|
| 6.544431400000001|         61985.76|
+------------------+-----------------+
only showing top 2 rows



In [121]:
df.selectExpr("*", "green_zone_part * 100 as green_zone_percent", "price_doc / 75 as price_usd").show(2)

+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+-----------------+
|          timestamp|full_sq|life_sq|floor|  sub_area|green_zone_part|indust_part|price_doc|green_zone_percent|        price_usd|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+-----------------+
|2014-03-27 00:00:00|     35|     19|    3|Ivanovskoe|    0.512707469| 1.69676E-4|  5300000|        51.2707469|70666.66666666667|
|2014-03-27 00:00:00|     46|     NA|   12|  Tverskoe|    0.065444314| 7.81528E-5|  4648932| 6.544431400000001|         61985.76|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+-----------------+
only showing top 2 rows



In [125]:
df.selectExpr("avg(price_doc)", "count(*)").show(2)

+----------------+--------+
|  avg(price_doc)|count(1)|
+----------------+--------+
|7123035.27773949|   30471|
+----------------+--------+



In [128]:
df.select("timestamp", lit(5).alias("Five")).show(2)

+-------------------+----+
|          timestamp|Five|
+-------------------+----+
|2014-03-27 00:00:00|   5|
|2014-03-27 00:00:00|   5|
+-------------------+----+
only showing top 2 rows



In [132]:
df.withColumn("more_green_than_indust", expr("green_zone_part > indust_part")).show(2)

+-------------------+-------+-------+-----+----------+---------------+-----------+---------+----------------------+
|          timestamp|full_sq|life_sq|floor|  sub_area|green_zone_part|indust_part|price_doc|more_green_than_indust|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+----------------------+
|2014-03-27 00:00:00|     35|     19|    3|Ivanovskoe|    0.512707469| 1.69676E-4|  5300000|                  true|
|2014-03-27 00:00:00|     46|     NA|   12|  Tverskoe|    0.065444314| 7.81528E-5|  4648932|                  true|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+----------------------+
only showing top 2 rows



In [133]:
df.withColumnRenamed("timestamp", "selling_date").columns

['selling_date',
 'full_sq',
 'life_sq',
 'floor',
 'sub_area',
 'green_zone_part',
 'indust_part',
 'price_doc']

In [134]:
df.drop("sub_area").columns

['timestamp',
 'full_sq',
 'life_sq',
 'floor',
 'green_zone_part',
 'indust_part',
 'price_doc']

In [135]:
df.withColumn("green_zone_percent", expr("green_zone_part * 100").cast("long")).show(2)

+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+
|          timestamp|full_sq|life_sq|floor|  sub_area|green_zone_part|indust_part|price_doc|green_zone_percent|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+
|2014-03-27 00:00:00|     35|     19|    3|Ivanovskoe|    0.512707469| 1.69676E-4|  5300000|                51|
|2014-03-27 00:00:00|     46|     NA|   12|  Tverskoe|    0.065444314| 7.81528E-5|  4648932|                 6|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+
only showing top 2 rows



In [183]:
df.select(lower(col("sub_area"))).show(2)

+---------------+
|lower(sub_area)|
+---------------+
|     ivanovskoe|
|       tverskoe|
+---------------+
only showing top 2 rows



## Filtering

In [136]:
df.filter(col("floor") > 5).count()

17030

In [142]:
df.where(col("floor") > 5).where(expr("full_sq > 100")).select(expr("avg(price_doc)").cast("int")).show()

+---------------------------+
|CAST(avg(price_doc) AS INT)|
+---------------------------+
|                   21172852|
+---------------------------+



In [170]:
df.select(round(df.price_doc / 1_000_000, 3).alias("price in mln rbl")).show(5)

+----------------+
|price in mln rbl|
+----------------+
|             5.3|
|           4.649|
|           6.296|
|            6.53|
|            10.3|
+----------------+
only showing top 5 rows



In [164]:
df.where(df.sub_area == "Kuncevo").show(2)

+-------------------+-------+-------+-----+--------+---------------+-----------+---------+
|          timestamp|full_sq|life_sq|floor|sub_area|green_zone_part|indust_part|price_doc|
+-------------------+-------+-------+-----+--------+---------------+-----------+---------+
|2014-03-27 00:00:00|    115|     55|    2| Kuncevo|    0.070662054|0.035145267| 19074000|
|2014-03-28 00:00:00|     76|     42|    6| Kuncevo|    0.070662054|0.035145267|  2000000|
+-------------------+-------+-------+-----+--------+---------------+-----------+---------+
only showing top 2 rows



In [165]:
df.where(instr(df.sub_area, "Kun") >= 1).where((col("full_sq") > 100) | (col("floor") > 30)).show(2)

+-------------------+-------+-------+-----+--------+---------------+-----------+---------+
|          timestamp|full_sq|life_sq|floor|sub_area|green_zone_part|indust_part|price_doc|
+-------------------+-------+-------+-----+--------+---------------+-----------+---------+
|2014-03-27 00:00:00|    115|     55|    2| Kuncevo|    0.070662054|0.035145267| 19074000|
|2014-04-15 00:00:00|    114|     54|   16| Kuncevo|    0.070662054|0.035145267| 19967500|
+-------------------+-------+-------+-----+--------+---------------+-----------+---------+
only showing top 2 rows



## Date and time

In [190]:
df.sample(fraction=0.0002).select(col("timestamp"), date_add(col("timestamp"), 5), datediff(current_date(), col("timestamp"))).show(5)

+-------------------+----------------------+-----------------------------------+
|          timestamp|date_add(timestamp, 5)|datediff(current_date(), timestamp)|
+-------------------+----------------------+-----------------------------------+
|2014-04-24 00:00:00|            2014-04-29|                               2439|
|2014-06-16 00:00:00|            2014-06-21|                               2386|
|2014-08-12 00:00:00|            2014-08-17|                               2329|
|2014-11-11 00:00:00|            2014-11-16|                               2238|
|2011-11-21 00:00:00|            2011-11-26|                               3324|
+-------------------+----------------------+-----------------------------------+
only showing top 5 rows



In [191]:
df.filter(col("timestamp") > "2014-04-24").count()

12675

## Nulls

In [200]:
df_with_null = df.na.replace(["NA"], [None], "life_sq")
df_with_null.show(2)

+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
|          timestamp|full_sq|life_sq|floor|  sub_area|green_zone_part|indust_part|price_doc|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
|2014-03-27 00:00:00|     35|     19|    3|Ivanovskoe|    0.512707469| 1.69676E-4|  5300000|
|2014-03-27 00:00:00|     46|   null|   12|  Tverskoe|    0.065444314| 7.81528E-5|  4648932|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
only showing top 2 rows



In [201]:
df_with_null.where(col("life_sq").isNotNull()).show(2)

+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
|          timestamp|full_sq|life_sq|floor|  sub_area|green_zone_part|indust_part|price_doc|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
|2014-03-27 00:00:00|     35|     19|    3|Ivanovskoe|    0.512707469| 1.69676E-4|  5300000|
|2014-03-27 00:00:00|     65|     43|   11|    Perovo|    0.065408826|0.225824954| 10300000|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
only showing top 2 rows



In [203]:
df_with_null.count(), df_with_null.na.drop().count()

(30471, 24088)

In [206]:
df_with_null.na.fill({"life_sq": 0}).show(2)

+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
|          timestamp|full_sq|life_sq|floor|  sub_area|green_zone_part|indust_part|price_doc|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
|2014-03-27 00:00:00|     35|     19|    3|Ivanovskoe|    0.512707469| 1.69676E-4|  5300000|
|2014-03-27 00:00:00|     46|      0|   12|  Tverskoe|    0.065444314| 7.81528E-5|  4648932|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
only showing top 2 rows



## Structs

In [209]:
df.selectExpr("(green_zone_part, indust_part) as area_parts", "*").limit(2).toPandas()

,area_parts,timestamp,full_sq,life_sq,floor,sub_area,green_zone_part,indust_part,price_doc
0,"(0.512707469, 0.000169676)",2014-03-27,35,19,3,Ivanovskoe,0.512707,0.000170,5300000
1,"(0.065444314, 7.81528e-05)",2014-03-27,46,NA,12,Tverskoe,0.065444,0.000078,4648932


In [213]:
df_with_struct = df.select(struct("green_zone_part", "indust_part").alias("area_parts"))
df_with_struct.limit(2).toPandas()

,area_parts
0,"(0.512707469, 0.000169676)"
1,"(0.065444314, 7.81528e-05)"


In [214]:
df_with_struct.select(col("area_parts").getField("green_zone_part")).show(2)

+--------------------------+
|area_parts.green_zone_part|
+--------------------------+
|               0.512707469|
|               0.065444314|
+--------------------------+
only showing top 2 rows



In [215]:
df_with_struct.select("area_parts.green_zone_part").show(2)

+---------------+
|green_zone_part|
+---------------+
|    0.512707469|
|    0.065444314|
+---------------+
only showing top 2 rows



In [216]:
df_with_struct.select("area_parts.*").show(2)

+---------------+-----------+
|green_zone_part|indust_part|
+---------------+-----------+
|    0.512707469| 1.69676E-4|
|    0.065444314| 7.81528E-5|
+---------------+-----------+
only showing top 2 rows



## Arrays

In [246]:
df_split = df.select(split(col("sub_area"), " ").alias("sub_area_split")).filter(size(col("sub_area_split")) > 1)
df_split.selectExpr("sub_area_split[1]").show(2)

+-----------------+
|sub_area_split[1]|
+-----------------+
|           Butovo|
|       Moskovskij|
+-----------------+
only showing top 2 rows



In [247]:
df_exploded = df_split.withColumn("id", monotonically_increasing_id()).withColumn("exploded", explode(col("sub_area_split")))
df_exploded.limit(5).toPandas()

,sub_area_split,id,exploded
0,"[Juzhnoe, Butovo]",0,Juzhnoe
1,"[Juzhnoe, Butovo]",0,Butovo
2,"[Poselenie, Moskovskij]",1,Poselenie
3,"[Poselenie, Moskovskij]",1,Moskovskij
4,"[Nagatinskij, Zaton]",2,Nagatinskij


In [249]:
# https://stackoverflow.com/questions/43357727/how-to-do-opposite-of-explode-in-pyspark
df_exploded.groupBy("id").agg(collect_list("exploded")).limit(2).toPandas()

,id,collect_list(exploded)
0,26,"[Poselenie, Moskovskij]"
1,29,"[Poselenie, Vnukovskoe]"


## Maps

In [253]:
df_map = df.select(create_map("sub_area", "price_doc").alias("m"))
df_map.limit(2).toPandas()

,m
0,{'Ivanovskoe': 5300000}
1,{'Tverskoe': 4648932}


In [255]:
df_map.selectExpr("m['Ivanovskoe']").show(2)

+-------------+
|m[Ivanovskoe]|
+-------------+
|      5300000|
|         null|
+-------------+
only showing top 2 rows



## Json

In [267]:
df_json = spark.range(1).selectExpr("""'{"key1": {"value1": [1, 2, 3]}, "key2": 5}' as json_str""")
df_json.collect()[0]['json_str']

'{"key1": {"value1": [1, 2, 3]}, "key2": 5}'

In [271]:
df_json.select(get_json_object(df_json.json_str, "$.key1.value1[2]")).show()

+-------------------------------------------+
|get_json_object(json_str, $.key1.value1[2])|
+-------------------------------------------+
|                                          3|
+-------------------------------------------+



In [277]:
df.selectExpr("(green_zone_part, indust_part) as area_parts").select(to_json(col("area_parts")).alias("json")).limit(1).collect()[0]["json"]

'{"green_zone_part":0.512707469,"indust_part":1.69676E-4}'

## UDF (User-Defined Functions)

In [278]:
def rub_to_usd(rub): return rub / 75
rub_to_usd(100)

1.3333333333333333

In [283]:
rub_to_usd_udf = udf(rub_to_usd) # register UDF
df.select("price_doc", rub_to_usd_udf(col("price_doc"))).show(2)

+---------+---------------------+
|price_doc|rub_to_usd(price_doc)|
+---------+---------------------+
|  5300000|    70666.66666666667|
|  4648932|             61985.76|
+---------+---------------------+
only showing top 2 rows



In [285]:
spark.udf.register("rub_to_usd", rub_to_usd)
df.selectExpr("price_doc", "rub_to_usd(price_doc)").show(2)

+---------+---------------------+
|price_doc|rub_to_usd(price_doc)|
+---------+---------------------+
|  5300000|    70666.66666666667|
|  4648932|             61985.76|
+---------+---------------------+
only showing top 2 rows



In [327]:
@udf('double')
def rub_to_usd_dec(rub): return rub / 75
df.select("price_doc", rub_to_usd_dec(col("price_doc"))).show(2)

+---------+-------------------------+
|price_doc|rub_to_usd_dec(price_doc)|
+---------+-------------------------+
|  5300000|        70666.66666666667|
|  4648932|                 61985.76|
+---------+-------------------------+
only showing top 2 rows



In [304]:
def rub_to_usd_pandas(rub: pd.Series) -> pd.Series:
    # https://stackoverflow.com/questions/58878848/java-lang-illegalargumentexception-when-applying-a-python-udf-to-a-spark-datafra
    # https://issues.apache.org/jira/browse/SPARK-29367
    #import os
    #os.environ['ARROW_PRE_0_15_IPC_FORMAT']='1'
    return rub / 75

rub_to_usd_pandas(pd.Series([100, 200, 300]))

0    1.333333
1    2.666667
2    4.000000
dtype: float64

In [317]:
rub_to_usd_pd_udf = pandas_udf(rub_to_usd_pandas, returnType=LongType())
df.select("price_doc", rub_to_usd_pd_udf(col("price_doc"))).show(2)

+---------+----------------------------+
|price_doc|rub_to_usd_pandas(price_doc)|
+---------+----------------------------+
|  5300000|                       70666|
|  4648932|                       61985|
+---------+----------------------------+
only showing top 2 rows



In [335]:
@pandas_udf('double', functionType=PandasUDFType.SCALAR)
def rub_to_usd_pd_dec(rub):
    return rub / 75
df.select("price_doc", rub_to_usd_pd_dec(col("price_doc"))).show(2)

+---------+----------------------------+
|price_doc|rub_to_usd_pd_dec(price_doc)|
+---------+----------------------------+
|  5300000|           70666.66666666667|
|  4648932|                    61985.76|
+---------+----------------------------+
only showing top 2 rows



## Statistical methods

In [143]:
df.select("sub_area").distinct().count()

146

In [181]:
df.stat.corr("full_sq", "price_doc")

0.3418404597538172

In [177]:
df.select(corr(unix_timestamp(col("timestamp")), col("price_doc"))).show()

+---------------------------------------------------------------+
|corr(unix_timestamp(timestamp, yyyy-MM-dd HH:mm:ss), price_doc)|
+---------------------------------------------------------------+
|                                            0.11675321295024581|
+---------------------------------------------------------------+



In [386]:
df.describe().toPandas()

,summary,full_sq,life_sq,floor,sub_area,green_zone_part,indust_part,price_doc
0,count,30471,30471,30471,30471,30471,30471,30471
1,mean,54.21426930524105,34.403271338425775,7.670802534318902,None,0.21892226384815047,0.11887130937167767,7123035.27773949
2,stddev,38.031487324105036,52.28573347591002,5.319988520945879,None,0.17509015954361357,0.11868751297633602,4780111.329633794
3,min,0,0,0,Ajeroport,0.001879375,0.0,100000
4,max,5326,NA,NA,Zjuzino,0.852922841,0.521867054,111111112


In [145]:
df.sample(withReplacement=False, fraction=0.0002, seed=42).show()

+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+
|          timestamp|full_sq|life_sq|floor|            sub_area|green_zone_part|indust_part|price_doc|
+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+
|2012-10-19 00:00:00|     53|     29|   11|             Mar'ino|    0.188712765|0.090799103|  7500000|
|2014-09-19 00:00:00|     64|     NA|    9|Poselenie Moskovskij|    0.492839845|0.075778755|  6004650|
|2013-09-10 00:00:00|     39|     18|   12|Pokrovskoe Stresh...|    0.183968555|0.150817539|  6650000|
|2012-03-21 00:00:00|     40|     18|   16|            Brateevo|    0.398794394|        0.0|  6600000|
+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+



In [148]:
data_frames = df.randomSplit([0.7, 0.2, 0.1], seed=42)
len(data_frames), data_frames[0].count(), data_frames[1].count(), data_frames[2].count()

(3, 21367, 6116, 2988)

In [149]:
data_frames[0].union(data_frames[1]).union(data_frames[2]).count()

30471

In [150]:
df.sort(col("price_doc").desc()).show(5)

+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+
|          timestamp|full_sq|life_sq|floor|            sub_area|green_zone_part|indust_part|price_doc|
+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+
|2012-05-10 00:00:00|     55|     31|   12|         Teplyj Stan|    0.426396297|0.002211665|111111112|
|2015-03-12 00:00:00|    220|    144|    3|         Presnenskoe|    0.068202173|0.042031587| 95122496|
|2013-04-19 00:00:00|    185|     66|   33|             Ramenki|    0.169625419|0.019508796| 91066096|
|2014-05-29 00:00:00|    184|     85|   10|             Ramenki|    0.169625419|0.019508796| 80777440|
|2013-03-06 00:00:00|    206|    128|    2|Pokrovskoe Stresh...|    0.183968555|0.150817539| 78802248|
+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+
only showing top 5 rows



## Aggregation

* https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html
* https://stackoverflow.com/questions/40006395/applying-udfs-on-groupeddata-in-pyspark-with-functioning-python-example
* https://stackoverflow.com/questions/48160252/user-defined-function-to-be-applied-to-window-in-pyspark
* https://danvatterott.com/blog/2018/09/06/python-aggregate-udfs-in-pyspark/
* https://stackoverflow.com/questions/61855832/remove-rows-from-pyspark-dataframe-by-type

In [289]:
df.select(count("sub_area"), countDistinct("sub_area"), approx_count_distinct("sub_area", 0.1)).show()

+---------------+------------------------+-------------------------------+
|count(sub_area)|count(DISTINCT sub_area)|approx_count_distinct(sub_area)|
+---------------+------------------------+-------------------------------+
|          30471|                     146|                            128|
+---------------+------------------------+-------------------------------+



In [291]:
df.groupBy("sub_area", "floor").count().show()

+--------------------+-----+-----+
|            sub_area|floor|count|
+--------------------+-----+-----+
|           Taganskoe|   10|    8|
|        Metrogorodok|    4|   11|
|          Nekrasovka|   18|    8|
|         Bogorodskoe|   10|    7|
|       Lomonosovskoe|   10|    4|
|             Zjuzino|   16|    3|
|          Ivanovskoe|   15|    1|
|Orehovo-Borisovo ...|   NA|    1|
|       Lomonosovskoe|   12|    1|
|Birjulevo Vostochnoe|    6|   26|
|Poselenie Shherbinka|    2|   29|
|            Tverskoe|   11|   26|
| Vostochnoe Degunino|    5|   15|
|Poselenie Desjono...|   10|   26|
|     Severnoe Butovo|    5|   14|
|  Severnoe Izmajlovo|    1|   19|
|            Ljublino|   12|   12|
|  Juzhnoe Medvedkovo|    9|   12|
|            Jasenevo|    6|   15|
|          Sokol'niki|    6|    4|
+--------------------+-----+-----+
only showing top 20 rows



In [293]:
df.groupBy("sub_area").agg(avg("price_doc"), first("green_zone_part")).show()

+--------------------+--------------------+-----------------------------+
|            sub_area|      avg(price_doc)|first(green_zone_part, false)|
+--------------------+--------------------+-----------------------------+
|  Nagatino-Sadovniki|   7430191.227848101|                  0.094680962|
|          Pechatniki|        5684482.8125|                  0.030696593|
|Poselenie Filimon...|   3374459.227822581|                  0.548991609|
|             Hovrino|   8546164.140449438|                  0.184282943|
|Orehovo-Borisovo ...|   6677480.558252427|                  0.387099854|
|     Juzhnoe Tushino|   7185202.171428571|                  0.222646321|
|         Presnenskoe|1.4032071415789474E7|                  0.068202173|
|           Basmannoe|1.1587719163265307E7|                  0.015233744|
|        Jaroslavskoe|           6335919.0|                  0.037753826|
|            Caricyno|   5840068.181818182|                  0.071119749|
|             Marfino|   6754174.58823

In [336]:
@pandas_udf(df.schema, functionType=PandasUDFType.GROUPED_MAP)
# Input/output are both a pandas.DataFrame
def subtract_mean(pdf):
    return pdf.assign(price_doc=pdf.price_doc - pdf.price_doc.mean())

df.groupBy("sub_area").apply(subtract_mean).select("sub_area", "price_doc").show(10)

+------------------+---------+
|          sub_area|price_doc|
+------------------+---------+
|Nagatino-Sadovniki|   369808|
|Nagatino-Sadovniki|  -130191|
|Nagatino-Sadovniki| -1630191|
|Nagatino-Sadovniki|  -780191|
|Nagatino-Sadovniki| -1230191|
|Nagatino-Sadovniki| -5430191|
|Nagatino-Sadovniki|  1469808|
|Nagatino-Sadovniki|  4413808|
|Nagatino-Sadovniki| -1430191|
|Nagatino-Sadovniki| -3630191|
+------------------+---------+
only showing top 10 rows



In [345]:
@pandas_udf(DoubleType(), functionType=PandasUDFType.GROUPED_AGG)
def non_life_sq(full_sq, life_sq):
    return (full_sq - life_sq).mean()

df.withColumn("life_sq_int", df.life_sq.cast(LongType())).na.drop().groupBy("sub_area").agg(non_life_sq("full_sq", "life_sq_int").alias("non_life_sq_mean")).show(5)

+--------------------+------------------+
|            sub_area|  non_life_sq_mean|
+--------------------+------------------+
|  Nagatino-Sadovniki|19.098039215686274|
|          Pechatniki|17.979166666666668|
|             Hovrino|21.594285714285714|
|Poselenie Filimon...| 5.853658536585366|
|Orehovo-Borisovo ...| 17.43915343915344|
+--------------------+------------------+
only showing top 5 rows



## Window functions

In [295]:
window = Window \
    .partitionBy("sub_area") \
    .orderBy("price_doc") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [299]:
df.select("sub_area",
    "price_doc",
    max("price_doc").over(window),  # max price so far
    rank().over(window),
    dense_rank().over(window)).limit(5).toPandas()

,sub_area,price_doc,max(price_doc) OVER (PARTITION BY sub_area ORDER BY price_doc ASC NULLS FIRST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW),RANK() OVER (PARTITION BY sub_area ORDER BY price_doc ASC NULLS FIRST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW),DENSE_RANK() OVER (PARTITION BY sub_area ORDER BY price_doc ASC NULLS FIRST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
0,Nagatino-Sadovniki,990000,990000,1,1
1,Nagatino-Sadovniki,990000,990000,1,1
2,Nagatino-Sadovniki,1000000,1000000,3,2
3,Nagatino-Sadovniki,1000000,1000000,3,2
4,Nagatino-Sadovniki,1250000,1250000,5,3


In [303]:
spark.sql("""
    SELECT sub_area,
           price_doc,
           dense_rank(price_doc) OVER (PARTITION BY sub_area ORDER BY price_doc ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
    FROM df
""").limit(5).toPandas()

,sub_area,price_doc,DENSE_RANK() OVER (PARTITION BY sub_area ORDER BY price_doc ASC NULLS FIRST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
0,Nagatino-Sadovniki,990000,1
1,Nagatino-Sadovniki,990000,1
2,Nagatino-Sadovniki,1000000,2
3,Nagatino-Sadovniki,1000000,2
4,Nagatino-Sadovniki,1250000,3


## Joins

In [379]:
# Data is from https://www.domofond.ru/city-ratings/moskva-c3584
header = ["District", "Overall", "Ecology", "Cleanness", "Services", "Neighbours", "Childhood", "Leasure", "Shops", "Transport", "Safety", "LivingCost"]
data = """Kurkino,1,1,1,1,1,1,1,4,1,1,1
Strogino,2,3,5,3,5,3,2,11,26,11,4
Mitino,3,9,3,5,13,4,3,6,8,13,16
Marfino,4,2,2,9,11,27,85,68,39,3,2
Troparevo-Nikulino,5,6,27,7,4,41,8,25,10,14,7
Southern Butovo,6,12,11,8,10,8,32,32,13,23,17
Chertanovo Severnoye,7,10,14,11,23,16,12,38,21,21,8
Severnoe Butovo,8,18,15,15,7,24,17,2,31,32,24
Krylatskoe,9,5,8,13,21,19,4,105,66,7,68
Horoshevskoe,10,66,7,34,6,9,16,91,5,4,13
Lianozovo,11,35,12,4,59,2,7,1,25,74,29
Severniy,12,19,6,12,27,26,69,48,14,35,5
Ramenki,13,46,9,17,8,47,50,98,12,2,22
Lomonosovsky,14,43,25,52,15,18,28,41,32,12,10
Vernadsky Avenue,15,13,16,16,32,43,33,82,75,8,20
Novo-Peredelkino,16,21,13,6,36,22,15,9,78,53,40
Babushkinsky,17,11,36,43,53,33,11,34,24,34,14
Orekhovo-Borisovo Severnoye,18,41,10,10,51,11,26,19,59,33,19
Yasenevo,19,4,50,44,26,6,22,43,34,63,57
Teplyj Stan,20,22,51,18,20,35,47,28,30,45,3
Sokolniki,21,44,97,38,3,28,5,42,23,28,6
Gagarinsky,22,33,18,59,17,39,10,62,2,26,71
Akademicheskoe,23,48,28,31,30,17,36,67,18,20,9
Severnoe Tushino,24,7,49,23,58,23,14,57,42,54,34
Konkovo,25,8,46,24,54,29,25,15,38,51,54
Golovinsky,26,14,77,40,22,45,49,31,44,30,26
Kosino-Ukhtomsky,27,107,4,2,28,13,23,16,3,16,27
Obruchevsky,28,37,68,37,9,12,6,21,92,59,78
Ostankino,29,55,42,26,19,55,45,73,74,9,21
Filevsky Park,30,20,52,35,41,90,46,58,6,31,18
Brateevo,31,87,32,20,57,7,9,5,36,40,60
Novokosino,32,98,31,14,25,5,29,3,33,24,64
Preobrazhenskoe,33,25,37,48,74,54,63,54,48,46,11
Zyuzino,34,31,63,30,55,30,61,72,45,38,36
Fili-Davydkovo,35,32,34,78,24,38,68,52,68,29,39
Juzhnoe Tushino,36,16,38,63,95,63,19,56,65,39,67
Severnoe Medvedkovo,37,45,47,61,70,20,39,12,47,66,35
Sokol,38,62,33,27,12,101,60,106,49,18,31
Levoberezhny,39,39,84,65,38,44,40,102,81,17,23
Rostokino,40,23,17,25,46,70,37,107,11,68,99
Hamovniki,41,83,26,79,18,52,20,111,85,6,42
Solntsevo,42,29,39,19,35,65,62,49,103,60,45
Bibirevo,43,40,76,32,68,25,21,14,89,81,41
Novogireevo,44,56,71,69,48,21,35,7,29,78,69
Tagansky,45,100,29,29,56,31,18,79,27,27,12
Moskvorechye-Saburovo,46,68,20,39,45,48,83,104,7,25,84
Donskoy,47,74,85,100,14,15,30,53,9,56,48
Ivanovskoe,48,27,59,64,97,14,75,13,41,72,37
Izmailovo,49,15,107,103,40,59,34,23,53,62,28
Yuzhnoye-Medvedkovo,50,53,70,47,73,77,24,51,37,58,53
Cheryomushki,51,49,30,21,42,64,84,84,71,37,97
Tverskoy,52,99,19,22,2,75,73,112,58,5,86
Kotlovka,53,36,22,53,78,62,71,37,16,47,113
Shchukino,54,26,44,58,89,81,43,93,76,42,72
Losinoostrovsky,55,17,43,51,61,49,88,35,91,69,93
Voikovsky,56,59,86,71,43,51,58,36,35,70,43
Nagatinsky Zaton,57,69,57,41,31,78,65,97,83,41,32
Nagatino-Sadovniki,58,70,56,90,99,36,31,26,22,50,52
Vostochnoe Izmajlovo,59,30,89,102,49,37,48,18,43,87,44
Otradnoe,60,78,35,36,87,32,57,8,52,90,56
Yaroslavl,61,34,54,70,66,53,89,22,70,67,85
Presnensky,62,93,45,67,16,85,52,109,40,10,79
Alekseyevsky,63,77,21,42,103,93,38,65,28,52,62
Zyablikovo,64,67,48,49,84,57,70,20,51,76,80
Sviblovo,65,28,81,94,83,106,55,46,4,73,47
Pokrovskoe-Streshnevo,66,50,91,83,37,56,79,85,46,71,66
Basmanny,67,91,72,74,47,86,51,92,15,44,33
Horoshevo-Mnevniki,68,72,58,55,64,69,53,55,97,49,87
Butyrsky,69,85,73,99,69,40,76,76,63,36,15
Koptevo,70,47,99,107,39,95,86,39,20,57,49
Sokolinaja Gora,71,64,109,86,33,108,27,64,19,65,59
Chertanovo Central'noe,72,65,55,46,102,66,56,71,56,93,74
Zamoskvorechye,73,97,40,50,90,102,42,108,73,22,50
Biryulyovo-Vostochnoye,74,60,53,72,92,50,87,30,105,82,77
Maryino,75,108,74,33,63,10,13,10,77,94,75
Nagorny,76,88,23,60,50,72,104,89,55,43,107
Orehovo-Borisovo Juzhnoe,77,82,41,28,93,68,95,27,90,85,82
Aeroport,78,84,103,108,34,92,82,90,82,19,46
Veshnjaki,79,61,88,84,75,34,44,29,94,106,96
Chertanovo Juzhnoe,80,75,66,56,94,71,78,33,96,92,73
Severnoe Izmajlovo,81,42,110,87,60,67,59,87,67,102,63
Savjolovskij,82,86,61,76,67,105,66,77,100,55,58
Perovo,83,54,94,96,82,87,67,69,62,86,90
Altuf'evskij,84,81,87,45,79,83,77,24,60,105,61
Kuncevo,85,52,78,106,65,80,90,86,87,75,100
Timirjazevskij,86,58,101,91,77,103,94,103,64,61,70
Hovrino,87,79,96,89,44,42,81,99,95,80,92
Mar'ina Roshha,88,104,64,92,71,84,41,17,84,64,88
Tekstil'shhiki,89,89,80,54,96,58,74,44,50,107,51
Vyhino-Zhulebino,90,90,98,98,52,46,72,40,102,88,76
Tsaritsyno,91,51,83,80,101,96,106,94,99,97,55
Lefortovo,92,102,90,95,81,94,80,59,54,79,25
Ochakovo-Matveevskoe,93,76,62,62,76,88,112,96,107,77,94
Beskudnikovskij,94,80,69,82,100,61,93,88,101,99,106
Vostochnoe Degunino,95,63,60,88,112,74,102,61,93,91,109
Mozhajskij,96,73,82,77,85,100,101,81,110,83,103
Begovoj,97,111,24,75,62,114,105,101,17,48,30
Bogorodskoe,98,57,104,113,104,98,103,66,72,89,89
Dorogomilovo,99,112,65,57,29,110,110,114,69,15,105
Gol'janovo,100,71,105,101,86,91,92,47,104,110,95
Danilovskij,101,105,92,93,91,111,91,80,61,84,65
Dmitrovskij,102,92,102,104,98,73,54,60,112,95,108
Metrogorodok,103,38,112,111,88,89,114,78,57,100,91
Vnukovo,104,24,67,73,72,113,111,113,113,104,102
Zapadnoe Degunino,105,96,108,81,80,99,100,45,109,103,98
Rjazanskij,106,95,106,105,105,76,99,50,88,111,83
Krasnosel'skij,107,103,93,85,111,109,97,95,80,109,101
Pechatniki,108,106,113,110,106,79,64,70,79,108,111
Kuz'minki,109,94,111,109,108,60,96,63,108,112,81
Nizhegorodskij,110,113,95,97,110,112,108,100,86,98,38
Birjuljovo Zapadnoe,111,101,75,68,113,82,109,75,111,114,112
Juzhnoportovyj,112,110,100,112,114,107,107,74,98,101,110
Ljublino,113,109,114,114,109,104,98,83,106,113,114
Nekrasovka,114,114,79,66,107,97,113,110,114,96,104"""

In [383]:
df_ranking = spark.createDataFrame([line.split(",") for line in data.split("\n")]).toDF(*header)
df_ranking.show(5)

+------------------+-------+-------+---------+--------+----------+---------+-------+-----+---------+------+----------+
|          District|Overall|Ecology|Cleanness|Services|Neighbours|Childhood|Leasure|Shops|Transport|Safety|LivingCost|
+------------------+-------+-------+---------+--------+----------+---------+-------+-----+---------+------+----------+
|           Kurkino|      1|      1|        1|       1|         1|        1|      1|    4|        1|     1|         1|
|          Strogino|      2|      3|        5|       3|         5|        3|      2|   11|       26|    11|         4|
|            Mitino|      3|      9|        3|       5|        13|        4|      3|    6|        8|    13|        16|
|           Marfino|      4|      2|        2|       9|        11|       27|     85|   68|       39|     3|         2|
|Troparevo-Nikulino|      5|      6|       27|       7|         4|       41|      8|   25|       10|    14|         7|
+------------------+-------+-------+---------+--

In [385]:
def get_districts(data_frame, column_name):
    return set(data_frame.select(column_name).distinct().toPandas()[column_name])

districts_sales = get_districts(df, "sub_area")
districts_ranking = get_districts(df_ranking, "District")
print(len(districts_sales), len(districts_ranking), len(set(districts_sales).intersection(set(districts_ranking))))

146 114 58


In [387]:
# TODO:
# 1. Demo join expression
# 2. Demo various join types
# 3. Demo join on complex types
# 4. Demo broadcast

In [389]:
# TODO:
# * SQL for each DataFrame API call
# * Correlated & uncorrelated predicate subqueries
# * Writing/reading Parquet files
# * Anything else from "Learning Spark"?
# * More on debugging & tuning

# Debugging

In [21]:
df.sort('timestamp').explain()

== Physical Plan ==
Sort [timestamp#11 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(timestamp#11 ASC NULLS FIRST, 200)
   +- FileScan csv [id#10,timestamp#11,full_sq#12,life_sq#13,floor#14,max_floor#15,material#16,build_year#17,num_room#18,kitch_sq#19,state#20,product_type#21,sub_area#22,area_m#23,raion_popul#24,green_zone_part#25,indust_part#26,children_preschool#27,preschool_quota#28,preschool_education_centers_raion#29,children_school#30,school_quota#31,school_education_centers_raion#32,school_education_centers_top_20_raion#33,... 268 more fields] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/anton/datasets/sberbank_russian_housing_market/train.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:int,timestamp:timestamp,full_sq:int,life_sq:string,floor:string,max_floor:string,materi...


In [287]:
#df.cache() #TODO good example and explain